## ДЗ №2. Матричные факторизации

#### В этой домашке вам предстоит реализовать некоторые базовые модели матричной факторизации

#### Дата выдачи: 17.02.25

#### Мягкий дедлайн: 02.03.25 23:59 MSK

#### Жесткий дедлайн: 09.03.25 23:59 MSK

В этом задании мы будем работать с классическим для рекоендательных систем датасетом [MovieLens 1M](https://grouplens.org/datasets/movielens/1m/). Датасет содержит рейтинги оценки для 4000 фильмов от 6000 пользователей. Более подробное описание можете найти на странице с датасетом и в README файле

In [2]:
pip install numpy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install typing

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip
!unzip ml-1m.zip
!cat ml-1m/README

--2025-02-22 22:55:01--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M   770KB/s    in 6.7s    

2025-02-22 22:55:13 (857 KB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C
SUMMARY

These files contain 1,000,209 anonymous ratings of approximately 3,900 movies 
made by 6,040 MovieLens users who joined MovieLens in 2000.

USAGE LICENSE

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be

In [6]:
import pandas as pd
import numpy as np
from typing import Union

In [7]:
df = pd.read_csv("ml-1m/ratings.dat", sep='::', names=['user_id', 'item_id', 'rating', 'timestamp'], engine='python')
df['datetime'] = pd.to_datetime(df['timestamp'], unit='s')
df.drop('timestamp', axis=1, inplace=True)
df.head()

,user_id,item_id,rating,datetime
0,1,1193,5,2000-12-31 22:12:40
1,1,661,3,2000-12-31 22:35:09
2,1,914,3,2000-12-31 22:32:48
3,1,3408,4,2000-12-31 22:04:35
4,1,2355,5,2001-01-06 23:38:11


In [8]:
value_counts = df['item_id'].value_counts()
filtered_values = value_counts[value_counts > 20].index
df = df[df['item_id'].isin(filtered_values)].copy()

In [9]:
train_end = '2000-12-01'
df_train = df[df['datetime'] < train_end].copy()
df_test = df[df['datetime'] >= train_end].copy()
df_train.shape, df_test.shape

((787420, 4), (207432, 4))

In [10]:
train_users = df_train['user_id'].unique()
train_items = df_train['item_id'].unique()

df_test = df_test[df_test['user_id'].isin(train_users)]
df_test = df_test[df_test['item_id'].isin(train_items)]
df_test.shape

(106471, 4)

In [11]:
pip install scikit-learn


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
from sklearn.preprocessing import LabelEncoder

user_le = LabelEncoder()
item_le = LabelEncoder()

df_train['user_id'] = user_le.fit_transform(df_train['user_id'])
df_train['item_id'] = item_le.fit_transform(df_train['item_id'])

df_test['user_id'] = user_le.transform(df_test['user_id'])
df_test['item_id'] = item_le.transform(df_test['item_id'])

In [13]:
df_train['user_id'].nunique(), df_train['user_id'].max()
df_train['item_id'].nunique(), df_train['item_id'].max()

(3010, np.int64(3009))

##### Задание 1. Напишем функцию, которая превратит датафрейм в матрицу интеракций. В функции df_to_matrix реализуйте функцию, которая принимает датафрейм и возвращает np.array матрицу интеракций. В функции df_to_coo реализуйте функцию, которая принимает датафрейм и возвращает разреженную матрицу интеракций в coo_array формате

In [14]:
df_train

,user_id,item_id,rating,datetime
100409,0,2994,3,2000-11-30 23:49:23
100411,0,929,4,2000-11-30 23:52:33
100412,0,567,4,2000-11-30 23:51:54
100415,0,3005,1,2000-11-30 23:58:06
100416,0,3006,4,2000-11-30 23:57:50
...,...,...,...,...
1000204,5364,814,1,2000-04-26 02:35:41
1000205,5364,817,5,2000-04-25 23:21:27
1000206,5364,478,5,2000-04-25 23:19:06
1000207,5364,819,4,2000-04-26 02:20:48


In [15]:
for _, row in df.iterrows():
    print(row)
    break

user_id                       1
item_id                    1193
rating                        5
datetime    2000-12-31 22:12:40
Name: 0, dtype: object


In [16]:
n_users = df_train['user_id'].nunique()
n_items = df_train['item_id'].nunique()

In [17]:
interactions_count = 0 # для разряженной матрицы


def df_to_matrix(df: pd.DataFrame) -> np.ndarray:
    result = np.zeros((n_users, n_items))
    global interactions_count
    for _, row in df.iterrows():
        interactions_count += 1
        result[row['user_id']][row['item_id']] += row['rating']
        
    print(n_users, n_items)
    return result #shape ~ [n_users, n_items]

In [18]:
interactions = df_to_matrix(df_train)
interactions.shape

5365 3010


(5365, 3010)

In [19]:
print(interactions_count)

787420


In [20]:
from scipy.sparse import coo_array

def df_to_coo(df: pd.DataFrame) -> coo_array:
    # global users, items, n_users, n_items
    data = np.ones(interactions_count)
    result = coo_array((df["rating"],(df["user_id"] , df["item_id"])), shape = (n_users,n_items))
    

    return result # coo_array

In [21]:
coo_interactions = df_to_coo(df_train)


In [22]:
print(interactions[0, 2994])

3.0


In [23]:
df_train[(df_train["user_id"] == 2369) & (df_train["item_id"] == 1203)]

,user_id,item_id,rating,datetime
495839,2369,1203,5,2000-09-29 01:02:22


In [24]:
assert (interactions != 0).sum() == df_train.shape[0]
assert interactions[0, 2994] == 3
assert interactions[2369, 1203] == 5
assert interactions[1557, 459] == 3
assert np.allclose(coo_interactions.toarray(), interactions)

##### Задание 2.1. Рассмотрим [SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition). Возьмите готовую реализуцию алгоритма из numpy.linalg или из scipy.linalg и примените алгоритм к матрицам интеракций, полученным в первом задании. Для работы со sparse матрицей обычная реализация svd не подойдет и нужно будет воспользоваться scipy.sparse.linalg.svds. Вам нужно разложить матрицу интеракций на 3 матрицы U, S, V, а затем перемножить их и восстановить изначальную матрицу. При полном разложении исходная матрица должна восстанавливаться максимально хорошо

In [25]:
pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [26]:
from scipy.sparse.linalg import svds
from scipy import linalg

def make_svd(interractions: Union[np.ndarray, coo_array], n_singular_values: int = -1):
    # функция должна работать и для полной матрицы и для sparse матрицы(вам поможет isinstance).
    # если n_singular_values = -1, то берем все сингулярные числа для полной матрицы
    # и все кроме одного сингулярного числа для coo-матрицы(иначе scipy.sparse.linalg.svds не будет работать)
    
    if isinstance(interractions, np.ndarray):
        U,s,V = linalg.svd(interractions, full_matrices=False)
        if n_singular_values != -1:
            U = U[:,:n_singular_values]
            S = np.diag(s)
            # print(S.shape)
            # print(len(s), n_singular_values+1)
            S = S[:n_singular_values, :n_singular_values]
            # print(S.shape)
            V = V[:n_singular_values , : ]
        else:
            S = np.diag(s)
        # print(S.shape)
        
    else:
        
        U,s,V = svds(interractions, k = n_singular_values)
        S = np.diag(s)
        

    #your code here
    
    return U, S, V

In [27]:
U, S, V = make_svd(interactions)
assert np.allclose(U @ S @ V, interactions)


In [ ]:
U.shape, S.shape, V.shape

((5365, 3010), (3010, 3010), (3010, 3010))

In [ ]:
U.shape, S.shape, V.shape

((5365, 3010), (3010, 3010), (3010, 3010))

In [28]:
U1, S1, V1 = make_svd(interactions, 10)
U, S, V = make_svd(coo_interactions, 10)
assert np.allclose(U1 @ S1 @ V1, U @ S @ V)

In [ ]:
U1.shape, S1.shape, V1.shape

((5365, 10), (10, 10), (10, 3010))

In [ ]:
U.shape , S.shape, V.shape

((5365, 10), (10, 10), (10, 3010))

##### Задание 2.2. Теперь попробуем сделать рекомендации с помощью SVD. Мы научились восстанавливать исходную матрицу с помощью разложения, теперь же мы хотим порекомендовать пользователю айтемы, которые будут для него максимально релевантны(в восстановленной матрице у них будет самый высокий скор). Для каждого пользователя нужно будет найти индексы айтемов, которые имеют максимальный скор. При этом стоит обратить внимание, что мы не хотим рекомендовать пользователю айтемы, с которыми он уже взаимодействовал

In [ ]:
# arr = enumerate(np.array([1,2,3]))
# arr[0]

In [ ]:
def make_svd_recommendations(interractions: Union[np.ndarray, coo_array], n_singular_values: int = -1, top_k: int = 100):
    # Возвращает матрицу вида n_users, top_k, то есть для каждого пользователя возвращаем индексы 
    # top_k самых релевантный айтемов среди тех с которыми он еще не взаимодействовал
    
    films_watched = np.zeros(interractions.shape) 
    films_watched = (interractions != 0)
    if isinstance(interractions, coo_array):
        films_watched = films_watched.toarray() # матрица фильмов, которые юзер смотрел
        
    U, S, V = make_svd(interractions, n_singular_values) # svd
    
    new_interaction = U @ S @ V # собираем обратно
    print(new_interaction)
    print()
    recommendations = np.zeros((interractions.shape[0], top_k))
    index = 0 # индекс по пользователям
    rec_ind = 0 # индекс по top-k у каждого пользователя

    cnt = 0
    for _, row in enumerate(new_interaction):
        print(type(row))
        dtype = [('ind', int), ('val', float)] # новый row, с учетом индексов изначальных
        new_row = np.array(row, dtype = dtype) # для отслеживания уже просмотренных фильмов
        
        
        if cnt < 1: # чекер
            print(row, "row")

        for i in range(len(row)): 
            new_row[i] = (i, row[i])

        if cnt < 1: # чекер
            print(new_row, "new_row")


        new_row = np.sort(new_row, order = 'val')[::-1] # соритирую скоры
        
        if cnt < 1: # чекер
            print( new_row, "sorted new row")
        
        cnt+=1 # чтобы проверить фильтр в первом обходе
        print(type(recommendations))
        print(type(films_watched))
        k = 0 
        while rec_ind < top_k:

            # if rec_ind < 1: # чекер
            #     print(films_watched[index][new_row[k][0]], new_row[k], "in recs loop")

            if films_watched[index][new_row[k][0]] == False: # если фильм не смотрели
                recommendations[index][rec_ind] = new_row[k][1] #
                rec_ind +=1
            k+=1
        index += 1

    return recommendations #shape ~ [n_users, top_k]

In [ ]:
recs = make_svd_recommendations(interactions, -1, 100)

[[ 8.51749227e-16  4.26897233e-13  7.80443418e-14 ... -4.16333634e-17
  -3.32373018e-15  3.99672835e-15]
 [-6.12444123e-15  1.14761716e-14 -9.56994900e-14 ... -6.93889390e-18
  -3.83373888e-16 -8.06646416e-16]
 [-5.06647675e-15  4.88166365e-14 -4.83191503e-14 ...  1.92207361e-15
   1.72648354e-15  5.89534931e-15]
 ...
 [-2.03092751e-15 -2.57346228e-15 -1.06165077e-15 ...  9.36750677e-17
   9.54097912e-16  7.76288755e-17]
 [-2.16428438e-15  1.34991844e-14  4.46387718e-15 ... -2.14238349e-16
  -1.76941795e-16 -1.17093835e-15]
 [ 3.00000000e+00  1.79308608e-14 -9.07563955e-15 ... -2.98372438e-16
  -6.03683770e-16 -2.45897053e-15]]

<class 'numpy.ndarray'>
[ 8.51749227e-16  4.26897233e-13  7.80443418e-14 ... -4.16333634e-17
 -3.32373018e-15  3.99672835e-15] row
[(   0,  8.51749227e-16) (   1,  4.26897233e-13) (   2,  7.80443418e-14)
 ... (3007, -4.16333634e-17) (3008, -3.32373018e-15)
 (3009,  3.99672835e-15)] new_row
[( 886,  5.00000000e+00) ( 846,  5.00000000e+00) (2602,  5.00000000e+00)

In [ ]:
recs.shape

(5365, 100)

In [ ]:
recs = make_svd_recommendations(interactions, -1, 100)
assert recs.shape == (interactions.shape[0], 100)




[[ 8.51749227e-16  4.26897233e-13  7.80443418e-14 ... -4.16333634e-17
  -3.32373018e-15  3.99672835e-15]
 [-6.12444123e-15  1.14761716e-14 -9.56994900e-14 ... -6.93889390e-18
  -3.83373888e-16 -8.06646416e-16]
 [-5.06647675e-15  4.88166365e-14 -4.83191503e-14 ...  1.92207361e-15
   1.72648354e-15  5.89534931e-15]
 ...
 [-2.03092751e-15 -2.57346228e-15 -1.06165077e-15 ...  9.36750677e-17
   9.54097912e-16  7.76288755e-17]
 [-2.16428438e-15  1.34991844e-14  4.46387718e-15 ... -2.14238349e-16
  -1.76941795e-16 -1.17093835e-15]
 [ 3.00000000e+00  1.79308608e-14 -9.07563955e-15 ... -2.98372438e-16
  -6.03683770e-16 -2.45897053e-15]]

<class 'numpy.ndarray'>
[ 8.51749227e-16  4.26897233e-13  7.80443418e-14 ... -4.16333634e-17
 -3.32373018e-15  3.99672835e-15] row
[(   0,  8.51749227e-16) (   1,  4.26897233e-13) (   2,  7.80443418e-14)
 ... (3007, -4.16333634e-17) (3008, -3.32373018e-15)
 (3009,  3.99672835e-15)] new_row
[( 886,  5.00000000e+00) ( 846,  5.00000000e+00) (2602,  5.00000000e+00)

In [ ]:
coo_interactions.shape

(5365, 3010)

In [ ]:
recs2 = make_svd_recommendations(coo_interactions, 1, 100)

[[3.92978664e-01 1.26881333e-01 6.97190477e-02 ... 1.48700880e-03
  2.27767622e-03 2.96119951e-02]
 [5.72804594e-01 1.84941873e-01 1.01622288e-01 ... 2.16745984e-03
  3.31993445e-03 4.31623607e-02]
 [6.30317563e-01 2.03511131e-01 1.11825766e-01 ... 2.38508563e-03
  3.65327550e-03 4.74961170e-02]
 ...
 [1.95094284e-01 6.29902461e-02 3.46120259e-02 ... 7.38225618e-04
  1.13075251e-03 1.47008770e-02]
 [9.82012713e-01 3.17063223e-01 1.74220632e-01 ... 3.71587997e-03
  5.69167543e-03 7.39972888e-02]
 [2.42438084e+00 7.82761765e-01 4.30113742e-01 ... 9.17371852e-03
  1.40515379e-02 1.82683592e-01]]

<class 'numpy.ndarray'>
[0.39297866 0.12688133 0.06971905 ... 0.00148701 0.00227768 0.029612  ] row
[(   0, 0.39297866) (   1, 0.12688133) (   2, 0.06971905) ...
 (3007, 0.00148701) (3008, 0.00227768) (3009, 0.029612  )] new_row
[( 211, 5.54783367e-01) ( 875, 5.54095993e-01) (2168, 5.14723127e-01) ...
 (2967, 5.07104258e-04) (2915, 4.16575087e-04) (1946, 1.38079058e-04)] sorted new row
<class 'nu

##### Задание 2.3. Теперь давайте посмотрим как будет зависеть качетво рекомендаций, от количества сингулярных чисел, которые мы возьмем в SVD разложении. Переберите n_singular_values из списка [1, 10, 50, 200, 1000] и посмотрите как будет изменяться метрика NDCG на тестовом датасете для таких рекомендаций и как будет меняться время вычисления. Для каждого графики зависимости метрики NDCG от n_singular_values и времени работы алгоритма от n_singular_values(Время работы будет меняться только для sparse-матрицы, стоит запускать алгоритм именно для нее)

In [ ]:

pip install -U matplotlib

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement matplotlib (from versions: none)
ERROR: No matching distribution found for matplotlib
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

def dcg_at_k(scores, k):
    scores = np.array(scores)[:k]
    return np.sum(scores / np.log2(np.arange(2, scores.size + 2)))

def ndcg_score(y_true, y_pred, k=10):
    indices = np.argsort(y_pred)[::-1]  # Сортируем индексы по убыванию предсказанных оценок
    ideal_indices = np.argsort(y_true)[::-1]  # Идеальное ранжирование
    
    dcg = dcg_at_k(y_true[indices], k)
    idcg = dcg_at_k(y_true[ideal_indices], k)
    
    return dcg / idcg if idcg > 0 else 0  # Нормализация

def plot_graphs(interactions: Union[np.ndarray, coo_array], top_k: int = 100):
    ndcg_scores = []
    computation_times = []
    singular_values_list = [1, 10, 50, 200, 1000]
    
    for n_singular_values in singular_values_list:
        start_time = time.time()
        
        # Используем make_svd_recommendations для получения рекомендаций
        recommendations = make_svd_recommendations(interactions, n_singular_values, top_k)
        
        # Предполагаем, что y_true - это фактические взаимодействия
        y_true = interactions.toarray() if isinstance(interactions, coo_array) else interactions
        ndcg = ndcg_score(y_true, recommendations, top_k)
        ndcg_scores.append(ndcg)
        
        computation_times.append(time.time() - start_time)
    
    # Построение графика NDCG
    plt.figure(figsize=(10, 5))
    plt.plot(singular_values_list, ndcg_scores, marker='o', linestyle='-', label='NDCG')
    plt.xlabel('Количество сингулярных чисел')
    plt.ylabel('NDCG')
    plt.title('Зависимость NDCG от количества сингулярных чисел')
    plt.legend()
    plt.grid()
    plt.show()
    
    # Построение графика времени работы
    plt.figure(figsize=(10, 5))
    plt.plot(singular_values_list, computation_times, marker='s', linestyle='-', color='r', label='Время работы (сек)')
    plt.xlabel('Количество сингулярных чисел')
    plt.ylabel('Время работы (сек)')
    plt.title('Зависимость времени работы от количества сингулярных чисел')
    plt.legend()
    plt.grid()
    plt.show()




ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
plot_graphs()

##### Задание 3.1. Перейдем к [ALS](http://yifanhu.net/PUB/cf.pdf). Возьмем реализацию iALS из библиотеки [implicit](https://benfred.github.io/implicit/api/models/cpu/als.html). Обучите ALS на нашем датасете, сделайте top_k рекомендации для юзеров из тестового датасета, и сравните метрики ALS с метриками, которые получились в SVD. Попробуйте перебрать гиперпараметры и найдите оптимальное число факторов, коэффициент alpha и коэффициент регуляризации.

In [ ]:
def make_als_recommendations(
    interractions: Union[np.ndarray, coo_array], 
    top_k: int = 100, 
    n_factors: int = 100,
    alpha: float = 1.0,
    regularization: float = 0.01,
):
    #your code here


    return recommendations #shape ~ [n_users, top_k]

In [ ]:
recs = make_als_recommendations(interractions)
assert recs.shape == (interractions.shape[0], 100)

##### Задание 3.2. Сделайте объяснение рекомендаций для нескольких юзеров(als.explain). Воспользуйтесь файлом movies.dat чтобы перейти от индексов фильмов к их названием

In [ ]:
#your code here

##### Задание 4. До этого мы работали с рейтингами, но как обсуждалось на лекции, implicit ALS отлично работает и с implicit фидбэком. Давайте попробуем преобразовать наш датасет(трейн и тест) следующим образом

1. Бинаризуем все рейтинги(заменим любую интеракцию пользователя на 1)
2. Заменим на 1 только рейтинги 4 и 5, а рейтинг ниже 4 заменим на 0
3. Заменим на 1 только рейтинги 4 и 5, а рейтинг ниже 4 заменим на -1
4. Заменим на 1 только рейтинги 4 и 5, а рейтинг ниже 4 заменим на -1 и добавим сглаживание по времени. То есть чем дальше была интеракция от максимальной даты трейна, тем с меньшим весом мы будем ее учитывать(например можно интеракции за последний месяц брать в исходном виде, и с каждым месяцем в прошлое умножать их на какой-нибудь коэффициент меньший 1). Таким образом более старые интеракции пользователя будут вносить меньший вклад в его интересы
5. Придумайте свой вариант(опционально)

Для каждой полученной матрицы обучите iALS и SVD и сравните их результаты между собой(преобразовывать нужно только обучающую выборку, тестовую оставляем неизменной)

In [ ]:
#your code here

##### Задание 5. iALS на numpy/torch. Давайте реализуем алгоритм iALS на нумпае или торче. Требуется реализовать алгорит, описанный в 4 части [статьи](http://yifanhu.net/PUB/cf.pdf). Обратите внимания на все оптимизации, которые они описывают в статье, чтобы сократить лишние вычисления. Hint: метрики у вашего алгоритма должны быть сравнимы с метриками ALS из библиотеки implicit

In [ ]:
class iALS:
    def __init__(self, n_factors: int = 100, alpha: float = 1.0, reg_coef = 0.01):
        #your code here

    def fit(self, interractions: np.ndarray, n_iterations: int 10):
        #your code here

    def predict(self, top_k: int = 100):
        # возвращает top-k айтемов для каждого юзера(айтемы с которыми юзер взаимодействовал не должны попасть в рекомендации)
        #your code here

        return predicts # shape ~ [n_users, top_k]